In [1]:
from i3d_inception import Inception_Inflated3d

In [64]:
import tensorflow as tf
import keras

from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import AveragePooling3D
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import Lambda
from keras.layers import GlobalAveragePooling3D

from keras import backend as K

In [65]:
NUM_FRAMES = 79
FRAME_HEIGHT = 224
FRAME_WIDTH = 224
NUM_RGB_CHANNELS = 3
NUM_FLOW_CHANNELS = 2
NUM_CLASSES = 2

# Transfer learning

In [66]:
rgb_model = Inception_Inflated3d(
                include_top=False,
                weights='rgb_imagenet_and_kinetics',
                input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS),
                classes=NUM_CLASSES)

In [67]:
rgb_model.summary()

Model: "i3d_inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 79, 224, 224 0                                            
__________________________________________________________________________________________________
Conv3d_1a_7x7_conv (Conv3D)     (None, 40, 112, 112, 65856       input_3[0][0]                    
__________________________________________________________________________________________________
Conv3d_1a_7x7_bn (BatchNormaliz (None, 40, 112, 112, 192         Conv3d_1a_7x7_conv[0][0]         
__________________________________________________________________________________________________
Conv3d_1a_7x7 (Activation)      (None, 40, 112, 112, 0           Conv3d_1a_7x7_bn[0][0]           
______________________________________________________________________________________

# Adding classification layer

In [68]:
from keras import models
from keras import optimizers

In [69]:
# Creating sequential model
model = models.Sequential()
model.add(rgb_model)

# Adding classification layers
dropout_prob = 0.0

model.add(Dropout(dropout_prob))
model.add(Conv3D(NUM_CLASSES, (1, 1, 1), 
                 strides = (1, 1, 1), 
                padding = 'same',
                use_bias = False,
                name = 'Conv3d_6a_1x1'))

num_frames_remaining = model.layers[-1].output_shape[1]

model.add(Reshape((num_frames_remaining, NUM_CLASSES)))

model.add(Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
                   output_shape=lambda s: (s[0], s[2])))

model.add(Activation('softmax', name = 'prediction'))

In [70]:
for layer in rgb_model.layers:
    layer.trainable = False

In [71]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i3d_inception (Functional)   (None, 9, 1, 1, 1024)     12294544  
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 1, 1, 1024)     0         
_________________________________________________________________
Conv3d_6a_1x1 (Conv3D)       (None, 9, 1, 1, 2)        2048      
_________________________________________________________________
reshape_2 (Reshape)          (None, 9, 2)              0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 2)                 0         
_________________________________________________________________
prediction (Activation)      (None, 2)                 0         
Total params: 12,296,592
Trainable params: 2,048
Non-trainable params: 12,294,544
______________________________________

# Data generators

In [72]:
import numpy as np
from keras.utils import Sequence
from keras.utils import np_utils
import os
import random
import math

In [73]:
class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args: 
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """
    def __init__(self, directory, batch_size=1, shuffle=True, data_augmentation=True, 
                 target_frames = 79, crop_dim = (224, 224), seed = None):
        # Initialize the params
        self.batch_size = batch_size
        self.directory = directory
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        self.target_frames = target_frames
        self.seed = seed
        self.crop_dim = crop_dim
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data() 
        # Print basic statistics information
        self.print_stats()
        return None
    
    def search_data(self):
        X_path = []
        Y_dict = {}
        # list all kinds of sub-folders
        self.dirs = sorted(os.listdir(self.directory))
        one_hots = np_utils.to_categorical(range(len(self.dirs)))
        for i,folder in enumerate(self.dirs):
            folder_path = os.path.join(self.directory,folder)
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path,file)
                # append the each file path, and keep its label  
                X_path.append(file_path)
                Y_dict[file_path] = one_hots[i]
        return X_path, Y_dict
    
    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = len(self.dirs)
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files,self.n_classes))
        for i,label in enumerate(self.dirs):
            print('%10s : '%(label),i)
        return None
    
    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)
    
    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y
    
    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def data_generation(self, batch_path):
        # load data into memory, you can change the np.load to any method you want        
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
    
    def dynamic_crop(self, video):
        video_dim = video.shape
        video_width = video_dim[2]
        video_height = video_dim[3]
        
        if self.data_aug:
            x_max = video_width - self.crop_dim[0]
            y_max = video_height - self.crop_dim[1]

            x = random.randint(0, x_max)
            y = random.randint(0, y_max)
            
        else:
            x_center = math.ceil(video_width/2)
            y_center = math.ceil(video_height/2)
            
            x = x_center - math.ceil(self.crop_dim[0]/2)
            y = y_center - math.ceil(self.crop_dim[1]/2)
                        
        return video[:,:,x:x+self.crop_dim[0],y:y+self.crop_dim[1],:]
        
    
    def frame_sampling(self, video):
        # get total frames of input video
        len_frames = video.shape[1]
        
        # If the video is shorter than needed
        if len_frames < self.target_frames:
            # Times the video need to be looped to get 64 frames
            times = self.target_frames//len_frames
            remainder = self.target_frames%len_frames
            # Creating new array to store cat video
            new_video = video
            
            # Repeat the video as many times as needed
            for n in range(1,times):
                new_video = np.concatenate((new_video, video), axis = 1)
            # Add part of the video if needed
            if remainder > 0:
                new_video = np.concatenate((new_video, video[:,:remainder,:,:]), axis = 1)
            
            return new_video
        
        # If the video is longer than needed
        elif len_frames > self.target_frames:
            # Set random start
            start_frame = random.randint(0,len_frames - self.target_frames)
            end_frame = start_frame + self.target_frames
            
            new_video = video[:,start_frame:end_frame,:,:]
            
            return new_video
        
        # If the video is fine
        elif len_frames == self.target_frames:
            return video
    
    def load_data(self, path):
        data = np.load(path)['arr_0']
    
        # Sampling frames
        if self.target_frames is not None:
            data = self.frame_sampling(video = data)
            
        data = self.dynamic_crop(data)

        return data[0]

In [74]:
batch_size = 8
path_train = '../datai3d/rgb/train/'
path_val = '../datai3d/rgb/validation/'

In [75]:
train_generator = DataGenerator(directory=path_train, 
                                batch_size=batch_size, 
                                data_augmentation=True)

validation_generator = DataGenerator(directory=path_val, 
                                batch_size=batch_size, 
                                data_augmentation=False)
                                #target_frames = None)

Found 1207 files belonging to 2 classes.
     Fight :  0
  NonFight :  1
Found 393 files belonging to 2 classes.
     Fight :  0
  NonFight :  1


In [76]:
#x, y = validation_generator.__getitem__(1)

In [77]:
#x.shape

# Training

In [79]:
# Parameters
epochs = 100
steps_per_epoch = train_generator.n_files//batch_size
validation_steps = validation_generator.n_files//batch_size

In [85]:
# Callbacks
filepath = 'checkpoints/weights_i3drgb.hdf5'

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                 factor=0.1,
                                                 patience=5, 
                                                 min_lr=0.001,
                                                verbose = 1)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                monitor='val_loss', 
                                                verbose=1, 
                                                save_best_only=True, 
                                                save_weights_only=False, 
                                                mode='auto', 
                                                save_freq='epoch')

earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                             restore_best_weights = True, 
                                             patience = 10, 
                                             min_delta = 0.01,
                                            verbose = 1)

callbacks = [reduce_lr, checkpoint, earlystop]

In [86]:
optimizer = keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9)

model.compile(optimizer=optimizer, loss='CategoricalCrossentropy', metrics=['accuracy'])

In [89]:
history = model.fit(
      train_generator,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps=validation_steps,
    callbacks = callbacks,
      verbose=1)

Epoch 1/100


ResourceExhaustedError:  OOM when allocating tensor with shape[8,40,112,112,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_2/i3d_inception/Conv3d_1a_7x7_conv/Conv3D (defined at <ipython-input-87-b2b390ed09ed>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_22619]

Function call stack:
train_function


In [88]:
keras.backend.clear_session()